In [ ]:
import pandas as pd
import numpy as np
file_path1 = "https://raw.githubusercontent.com/harshitcodes/tmdb_movie_data_analysis/refs/heads/master/tmdb-5000-movie-dataset/tmdb_5000_credits.csv"
file_path2 = 'https://raw.githubusercontent.com/harshitcodes/tmdb_movie_data_analysis/refs/heads/master/tmdb-5000-movie-dataset/tmdb_5000_movies.csv'

In [ ]:
credits = pd.read_csv(file_path1)
movies = pd.read_csv(file_path2)

In [ ]:
movies.head(5)

In [ ]:
credits.head(1)

In [ ]:
credits.head(1).cast.values

In [ ]:
movies = movies.merge(credits, on = 'title')

In [ ]:
movies.head(1)

In [ ]:
movies.info()

In [ ]:
movies['tagline']

In [ ]:
movies = movies[['genres', 'keywords', 'overview', 'title', 'movie_id', 'cast', 'crew']]

In [ ]:
movies.head(1)

In [ ]:
movies.shape

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
import ast

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies.cast.values[1]

In [ ]:
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 3:
            l.append(i['name'])
            counter += 1
        else:
            break
    return l

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies.crew[0]

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies

In [ ]:
movies.cast.values[0]

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [ ]:
movies.columns

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x : [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x : [i.replace(' ', '') for i in x])
movies

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['movie_id', 'title', 'tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:' '.join(x))

In [ ]:
new_df.tags[0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
cv.get_feature_names_out()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return ' '.join(l)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
print(cv.get_feature_names_out())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x: x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)
    

In [ ]:
recommend('Batman')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df, open('movies.pkl', 'wb'))

In [ ]:
new_df

In [ ]:
new_df.to_dict()

In [ ]:
pickle.dump(new_df.to_dict(), open('movie_dict.pkl', 'wb'))

In [ ]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))